In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from collections import defaultdict

In [ ]:
# Fungsi ini mendeteksi wajah dan menghitung Mouth Aspect Ratio
def calculate_mar(image, face_mesh):
    # Konversi gambar BGR ke RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    
    # Indeks landmark untuk bibir bagian dalam
    lip_indices = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324]
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Ambil koordinat titik-titik vertikal dan horizontal
            p_vertical1 = face_landmarks.landmark[13]
            p_vertical2 = face_landmarks.landmark[14]
            p_horizontal1 = face_landmarks.landmark[78]
            p_horizontal2 = face_landmarks.landmark[308]

            # Hitung jarak Euclidean
            vertical_dist = np.linalg.norm([p_vertical1.x - p_vertical2.x, p_vertical1.y - p_vertical2.y])
            horizontal_dist = np.linalg.norm([p_horizontal1.x - p_horizontal2.x, p_horizontal1.y - p_horizontal2.y])
            
            if horizontal_dist > 0:
                mar = vertical_dist / horizontal_dist
                return mar
    return 0.0 # Kembalikan 0 jika tidak ada wajah terdeteksi

In [ ]:
def group_files_by_subject(folder_path):
    files = os.listdir(folder_path)
    groups = defaultdict(list)
    for f in files:
        if '-' in f and f.endswith(('.jpg', '.png')):
            subject_id = f.split('-')[0]
            groups[subject_id].append(f)
    # Urutkan file di dalam setiap grup berdasarkan nomor frame
    for subject_id, file_list in groups.items():
        file_list.sort(key=lambda x: int(x.split('-')[1].split('.')[0]))
    return groups

In [ ]:
DATA_DIR = "YawDD dataset/"
YAWN_FOLDER = os.path.join(DATA_DIR, "yawn")
NO_YAWN_FOLDER = os.path.join(DATA_DIR, "no_yawn")

# Inisialisasi MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

all_features = []

In [ ]:
# Label 0 untuk no_yawn, Label 1 untuk yawn
for label, folder_path in enumerate([NO_YAWN_FOLDER, YAWN_FOLDER]): 
    print(f"Memproses folder: {os.path.basename(folder_path)} (Label: {label})")
    
    subject_groups = group_files_by_subject(folder_path)
    
    for subject_id, clip_files in subject_groups.items():
        mar_values_for_clip = []
        
        for file_name in clip_files:
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path)
            if image is not None:
                mar = calculate_mar(image, face_mesh)
                mar_values_for_clip.append(mar)
        
        if mar_values_for_clip:
            features = {
                "subject_id": f"{os.path.basename(folder_path)}_{subject_id}",
                "mar_mean": np.mean(mar_values_for_clip),
                "mar_max": np.max(mar_values_for_clip),
                "mar_std": np.std(mar_values_for_clip),
                "label": label
            }
            all_features.append(features)

Memproses folder: no_yawn (Label: 0)
Memproses folder: yawn (Label: 1)


In [ ]:
# Tutup objek face_mesh setelah selesai
face_mesh.close()

df = pd.DataFrame(all_features)
df.to_csv("yawn_features_final.csv", index=False)

print("\n=======================================================")
print("File 'yawn_features_final.csv' berhasil dibuat.")
print("Dataset siap untuk dilatih dengan model SVM.")
print("=======================================================")
print(df.head())


File 'yawn_features_final.csv' berhasil dibuat.
Dataset siap untuk dilatih dengan model SVM.
   subject_id  mar_mean   mar_max   mar_std  label
0   no_yawn_1  0.053303  0.179509  0.058467      0
1  no_yawn_10  0.080693  0.214398  0.043297      0
2  no_yawn_11  0.027227  0.038065  0.007924      0
3  no_yawn_12  0.050905  0.068027  0.009067      0
4  no_yawn_13  0.016059  0.022436  0.004218      0
